In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import collections 
import Counter
from wordcloud import WordCloud


In [ ]:
np.random.seed()
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize':(6,6)})
sns.set(font_scale=0.75)
pd.set_option('display max columns', None)
print(os.getcwd())

PATH = '../Python Scripts/pre.xlxs'

In [ ]:
def strip_words(row, unwanted_words):
    for col in ['summary', 'Details', 'resolution']:
        if isinstance(row[col], str):
            words = row[col].split()
            row[col] = ' '.join([item for item in words if item not in unwanted_words])
        else:
            row[col] = ''
    return row

In [ ]:
df = pd.read_excel(PATH)
df = pd.drop_duplicates()

patterns = ['[\w\.-]+@[\w\.-]+', 'to:[^\r\n]*([^\r\n]+)', 'from:[^\r\n]*([^\r\n]+)', 'cc:[^\r\n]*([^\r\n]+)', '[^a-zA-Z\r\n/_]']
unwanted_words = ['that', 'when', 'have', 'but', 'was', 'need', 'wo', 'incinc', '-', 'a', 'so', 'can', 'cgi', 'incident', 'support', 'to', 'and', 'is', 'on', 'in', 'the', 'for', 'inc', 'ph', 'not', 'philippine', 'india', 'us', 'new', 'issue', 'sr', 'pc', 'of', 'i', 'my', 'unable', 'nan', 'nan nan', 'com', 'it', 'you', 'this', 're', 'with', 'regards', 'your', 'frojm', 'as', 'be', 'as', 'cc', 'thanks', 'b', 'or', 'hi', 'subject', 'already', 'will', 'please', 'laptop']

# Remove email address and non-common symbols. Replace carriage return/new line, "-" and "/", with spaces
    df['summary'] = df['summary'].str.lower().replace(f'{pattern}', '', regex=True)
    df['resolution'] = df['resolution'].str.lower().replace(f'{pattern}', '', regex=True)
    df['Details'] = df['Details'].str.lower().replace(f'{pattern}', '', regex=True)

df['summary'] = df['summary'].str.lower().replace(r'[-/\n\r]', '', regex=True)
df['resolution'].fillna('', regex=True)
df['resolution'] = df['resolution'].str.lower().replace(r'[-/\n\r]', '', regex=True)
df['Details'] = df['Details'].str.lower().replace(r'[-/\n\r]', '', regex=True)

# Remove stopwords
df = df.apply(lambda row: strip_words(row, unwanted_words), axis=1)

df.loc[df['last resolve date'].isna(), 'Resolution time(days)'] = np.nan
df['Resolution Time'] = pd.cut(df['Resolution time(days)'], [0, 60, 90, 120, 365, 730, 2300], labels=['1-2 months', '2-3 months', '3-4 months', '4-12 months', '12-24 months', '24 months++'])

df['Year'] = df['submit date'].dt.strftime('%Y').astype(int)

df.loc[~df['id'].isna() & df['id'].str.contains('INS'), 'Type'] = 'Incident'
df.loc[~df['id'].isna() & df['id'].str.contains('WO'), 'Type'] = 'Work Order'
df.sample()

In [ ]:
df.value_counts('Year')

In [ ]:
df = df.loc[df['Year']>2020]

In [ ]:
df.loc[(df['Year']==2021) & (~df['Resolution time(days)'].isna())]

In [ ]:
sns.boxplot(df.loc[~df['Resolution time(days)'].isna()], y= 'Resolution time(days)', x='Year')

plt.title('Distribution of Resolution Time (days)')
plt.show()

### Figure 1 above
#### 2021
In the year 2021, there were around 1,019 cases with an average resolution time of approximateley 648 days. The resolution time varies moderately, with most cases falling between 605 and 697 days.
#### 2022
In 2022, thw average resolution time decreased to about 437 days for approximately 4, 474 cases. The range of resolution time also increased, with cases spanning from 240 to 603 days.
#### 2023
In 2023, the resolution time further reduced to around 134 days for approximately 2,817 cases. The variability in resolution time remained moderate, with cases being resolve within 28 to 234 days on average.

In [ ]:
df_plot = df.groupby(['Year', 'Resolution Time']).size().reset_index().pivot(columns='Resolution Time', index='Year', values=0).plot(kind='bar')

plt.title('Resolution Time (Months) Over the Years')
plt.show()

df_plot = df.groupby(['Year', 'Resolution Time']).size().reset_index().pivot(columns='Resolution Time', index='Year', values=0)

# Display the distribution of resolution times over the years using a bar plot
print("Resolution Time (Months Over the Years:)")
print(df_plot)

In [ ]:
sns.countplot(df, y='Year', hue='Type')
plt.title('Number of Tickets')
plt.show()

In [ ]:
sns.boxplot(df.loc[~df['Resolution time(days)'].isna()], y='Resolution time(days)', hue='Type', x='Year')

plt.title('Distribution of Resolution Time (days)')
plt.show()

In [ ]:
df['Status_2'] = df['status']
df.loc[df['status'].isin(['Closed', 'Completed', 'Resolved']), 'Status_2'] = 'Done'

In [ ]:
p_table = pd.pivot_table(df, values-'id', index='priority', columns='Status_2', aggfunc='count')

sns.heatmap(p_table, square=True, annot=True, fmt='.0f', cbar=False)
plt.title('Ticket Priority vs Status')
plt.show()

In [ ]:
sns.boxplot(df.loc[~df['Resolution time(days)'].isna()], y='Resolution time(days)', hue='priority', x='Year')

plt.title('Distribution of Resolution Time (days)')
plt.show()

In [ ]:
sns.boxplot(df.loc[~df['Resolution time(days)'].isna()], y='Resolution time(days)', x='priority', hue='assigned support group')

plt.title('Distribution of Resolution Time (days)')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='assigned support group', hue='Type')
plt.title("Assigned Support Group Counts by Type")
plt.xlabel("Assigned Support Group")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.legend(title='Type')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
import math
def create_wordclouds(df, cat_col, text_col, num_cols=2):
    df = df.loc[~df[cat_col].isna()]
    CATS = list(df[cat_col].isna())
    if len(CATS) <= num_cols:
        num_cols = len(CATS)
    num_rows = math.cell(len(CATS)/num_cols)

    fig, axes = plt.subplot(num_rows, num_cols, figsize=(4*num_cols, 3*num_rows))  # Adjust figsize as needed
    fig.subplots_adjust(hspace=0.1)  # Adjust vertical spacing between subplots

    for row in range(num_rows):
        for col in range(num_cols):
            idx = col+(num_cols*row)
            if idx < len(CATS):
                ax = axes[row, col] if num_rows > 1 else axes[col]  # Adjust indexing for single-row grids
                text = df.loc[df[cat_col]==CATS[idx], text_col].to_list()
                text = ' '.join([str(t) for t in text])

                try:
                    workloud = WordCloud().generate(text=text)
                    # Display the generated image:
                    ax.imshow(workloud, interpolation='bilinear')
                    ax.set_title(CATS[idx])
                except:
                    pass
            ax.axis("off")


    # Adjust layout and show plot
    plt.suptitle(f'Most common terms in {text_col}')
    plt.tight_layout()
    plt.show()

In [ ]:
import math
def create_barcharts(df, col_name, cat_col, num_cols=2):
    df = df.loc[~df[cat_col].isna()]
    CATS = list(df[cat_col].isna())
    if len(CATS) <= num_cols:
        num_cols = len(CATS)
    num_rows = math.cell(len(CATS)/num_cols)

    fig, axes = plt.subplot(num_rows, num_cols, figsize=(4*num_cols, 3*num_rows))  # Adjust figsize as needed
    fig.subplots_adjust(hspace=0.1)  # Adjust vertical spacing between subplots

    for row in range(num_rows):
        for col in range(num_cols):
            idx = col+(num_cols*row)
            if idx < len(CATS):
                ax = axes[row, col] if num_rows > 1 else axes[col]  # Adjust indexing for single-row grids
                df.loc[df[cat_col]==CATS[idx], 'Count'].sort_values(ascending=False).plot(kind='barh', ax=ax)
                ax.set_title(CATS[idx])


    # Adjust layout and show plot
    plt.suptitle(f'Most common terms in {col_name} by {cat_col}')
    plt.tight_layout()
    plt.show()

In [ ]:
def get_freq_words(df, col_name,top_k=10):
    col_values = df[col_name].astype(str).str.cat(sep=' ')
    top_words = pd.DataFrame(Counter(col_values.split()).most_common(top_k), columns=['Word', 'Count'])

def get_words_cat(df, cat_col, col_name, top_k=10):
    top_words = pd.DataFrame()
    for cat in list(df[cat_col].unique()):
        by_cat = get_freq_words(df.loc[df[cat_col]==cat], col_name)
        by_cat[cat_col] = cat
        top_words = pd.concat([top_words, by_cat])
    return top_words

In [ ]:
create_workclouds(df, 'Type', 'summary')

In [ ]:
create_workclouds(df, 'Type', 'Details')

In [ ]:
create_workclouds(df, 'Year', 'summary', num_cols=3)

In [ ]:
create_workclouds(df, 'Year', 'Details', num_cols=3)

### Figure 12 above
The most common terms or words in the 'Details' column categorized by year: 2021, 2022, 2023

In [ ]:
create_workclouds(df, 'priority', 'summary', num_cols=3)

### Figure 13 above
The most common terms or words in the 'summary' column categorized by its 'priority': Low, Medium, High, Critical

In [ ]:
create_workclouds(df, 'priority', 'Details', num_cols=3)

### Figure 14 above
The most common terms or words in the 'Details' column categorized by its 'priority': Low, Medium, High, Critical

In [ ]:
create_workclouds(df, 'status', 'summary', num_cols=3)

### Figure 15 above
The most common terms or words in the 'summary' column categorized by its 'status': Pending, Assigned, Completed, Closed, Cancelled, Resolved, In-Progress

In [ ]:
create_workclouds(df, 'status', 'Details', num_cols=3)

### Figure 16 above
The most common terms or words in the 'Details' column categorized by its 'status': Pending, Assigned, Completed, Closed, Cancelled, Resolved, In-Progress

In [ ]:
create_workclouds(df, 'assigned support group', 'summary', num_cols=3)

In [ ]:
create_workclouds(df, 'assigned support group', 'Details', num_cols=3)

In [ ]:
top_words = get_freq_words(df, 'summary',20)
top_words['Count'].sort_values(ascending=True).plot(kind='barh')

plt.title('Most frequent terms in summary')
plt.show()

In [ ]:
top_words = get_freq_words(df, 'Details',20)
top_words['Count'].sort_values(ascending=True).plot(kind='barh')

plt.title('Most frequent terms in Details')
plt.show()

### Figure 20 above
the bargraph shows the most common words found in the 'Details' column

In [ ]:
for word in top_words.index.to_list():
    avg_time = df.loc[df['summary'].str.contains(word), 'Resolution time(days)'].mean()
    top_words.loc[top_words.index == word, 'Resolution Time (mean)'] = avg_time

top_words['Resolution Time (mean)'].sort_values().plot(kind='barh')

plt.title('Mean Resolution Time (days) of Most Frequent Terms in summary')
plt.show()

### Figure 21 above
the bargraph shows the most common words found in the 'summary' column with their average 'Resolution time(days)' in the X-axis.

In [ ]:
for word in top_words.index.to_list():
    non_null_rows = df[df['Detailsa'].str.contains(word) & df['Resolution time(days)'].notnull()
    avg_time = non_null_rows['Resolution time(days)'].mean
    top_words.loc[top_words.index == word, 'Resolution Time (mean)'] = avg_time

top_words['Resolution Time (mean)'].sort_values().plot(kind='barh')

plt.title('Mean Resolution Time (days) of Most Frequent Terms in Details')
plt.show()

### Figure 22 above
the bargraph shows the most common words found in the 'Details' column with their average 'Resolution time(days)' in the X-axis.

In [ ]:
top_words_year = top_words_cat(df, 'Year', 'summary' ,20)
create_barcharts(top_words_year, 'summary', 'Year')

### Figure 23 above
the bargraph shows the most common words found in the 'summary' column sorted by their count or number of occurence, and categorized by Year: 2021, 2022, 2023

In [ ]:
top_words_year = top_words_cat(df, 'Year', 'Details' ,20)
create_barcharts(top_words_year, 'Details', 'Year')
# 'that', 'when', 'have', 'but', 'was', 'need'

### Figure 24 above
the bargraph shows the most common words found in the 'Details' column sorted by their count or number of occurence, and categorized by Year: 2021, 2022, 2023

In [ ]:
top_words_priority = top_words_cat(df, 'Year', 'priority' ,20)
create_barcharts(top_words_year, 'summary', 'priority')

### Figure 25 above
the bargraph shows the most common words found in the 'summary' column sorted by their count or number of occurence, and categorized by 'priority': Low, Medium, High, Critical

In [ ]:
top_words_priority = top_words_cat(df, 'priority', 'Details' ,20)
create_barcharts(top_words_year, 'Details', 'priority')

### Figure 26 above
the bargraph shows the most common words found in the 'Details' column sorted by their count or number of occurence, and categorized by 'priority': Low, Medium, High, Critical

In [ ]:
df['Year'] = df['submit date'].dt.year

yearly_counts = df.groupby('Year')['submit date'].count()

plt.figure(figsize=(12, 8))

num_years = len(yearly_counts)
cols = 2
rows = (num_years + cols - 1) // cols

for i, (year, count) in enumerate(yearly_counts.items(), start=1):
    plt.subplot(rows, cols, i)
    plt.histplot(df[df['Year'] == year]['submit date'], bins=12)  # Adjust bins as needed
    plt.title(f"Year {year} - Total per Month: {count}")
    plt.xlabel("Month")
    plt.ylabel("Count")
    plt.xticks(rotation=45)


plt.tight_layout()
plt.show()

In [ ]:
df.to_excel('explore.xlxs', index=False)